In [1]:
import threading
import os
import cv2
import face_recognition
import numpy as np
import tkinter as tk
from tkinter import scrolledtext
from PIL import Image, ImageTk
from datetime import datetime
import locale
import keyboard
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Đặt lại môi trường locale để hỗ trợ Unicode
locale.setlocale(locale.LC_ALL, 'vi_VN.UTF-8')

'vi_VN.UTF-8'

In [2]:
path = 'student_images'  # Thay đổi thành đường dẫn thư mục chứa hình ảnh 

In [3]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 128*128
NUM_CLASSES = len(path)
BATCH_SIZE = 256*torch.cuda.device_count()
DEVICE = 'cuda:0' # default GPU device
GRAYSCALE = False

In [4]:
images = []
classNames = []
classInfo = {}  # Từ điển để lưu thông tin từ tệp .txt
classNames1 = []
# Duyệt qua các thư mục và tệp tin trong `path`
for root, dirs, files in os.walk(path):
    for cl in dirs:  # Duyệt qua các thư mục con
        class_path = os.path.join(root, cl)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            if img_name.endswith(('png', 'jpg', 'jpeg')):
                curImg = cv2.imread(img_path)
                if curImg is not None:
                    images.append(curImg)
                    txt_path = os.path.join(class_path, cl + '.txt')
                    if os.path.exists(txt_path):
                        with open(txt_path, 'r', encoding='utf-8') as file:
                            content = file.read().strip().splitlines()
                            name = content[0].strip()  # Lấy tên từ dòng đầu tiên
                            info = '\n'.join(content[1:]).strip()  # Lấy thông tin từ các dòng còn lại
                            classNames.append(cl)
                            classNames1.append(name)
                            classInfo[name] = info
                    else:
                        print(f"Error: Could not find the text file {txt_path}")
                else:
                    print(f"Error: Could not load image {img_path}")

# Hiển thị số lượng hình ảnh đã được nạp
print(f"Loaded {len(images)} images and corresponding names and info.")

Loaded 37 images and corresponding names and info.


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import numpy as np
import torch
from torchvision import transforms
import cv2

# Đảm bảo rằng ảnh có cùng kích thước
def resize_image(image, size=(224, 224)):
    """Resize image to the given size."""
    return cv2.resize(image, size)

# Chuyển đổi dữ liệu thành numpy array
resized_images = [resize_image(img) for img in images]
images_array = np.array(resized_images)
classNames_array = np.array(classNames)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(images_array, classNames_array, test_size=0.2, random_state=42)

# Fit LabelEncoder với toàn bộ dữ liệu
all_labels = np.concatenate([y_train, y_test])
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Chuyển đổi nhãn
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# One-hot encoding
y_train_cat = to_categorical(y_train_encoded)
y_test_cat = to_categorical(y_test_encoded)

# Chuyển đổi dữ liệu thành tensor nếu bạn đang dùng PyTorch
transform = transforms.Compose([
    transforms.ToTensor(),  # Chuyển đổi ảnh thành tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize nếu cần
])

X_train_tensor = torch.stack([transform(image) for image in X_train])
X_test_tensor = torch.stack([transform(image) for image in X_test])

# Chuyển dữ liệu thành dạng Tensor
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

# Hiển thị số lượng hình ảnh đã được nạp
print(f"Number of training images: {len(X_train)}")
print(f"Number of test images: {len(X_test)}")


Number of training images: 29
Number of test images: 8


In [6]:
# Tạo DataLoader cho tập huấn luyện và tập kiểm tra
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Định nghĩa mô hình ResNet-34
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes, grayscale):
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1, padding=2)
        self.fc = nn.Linear(512 * 5 * 5, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        print("Shape before flattening:", x.shape)  # In kích thước tensor ở đây
        x = x.view(x.size(0), -1)  # Flatten
        print("Shape after flattening:", x.shape)  # In kích thước tensor sau khi flatten
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

def resnet34(num_classes, grayscale):
    """Constructs a ResNet-34 model."""
    model = ResNet(block=BasicBlock, layers=[3, 4, 6, 3], num_classes=num_classes, grayscale=grayscale)
    return model



In [8]:
# Khởi tạo mô hình ResNet-34
from torchvision import datasets, transforms
data_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('student_images', transform=transform),
    batch_size=1,
    shuffle=True
)

# Xác định số lớp
num_classes = len(data_loader.dataset.classes)
print(f"Number of classes: {num_classes}")
grayscale = False  # Thay đổi tùy theo dữ liệu của bạn
model = resnet34(num_classes, grayscale)

# Định nghĩa criterion và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Kỹ thuật giảm tốc độ học
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Callback: Lưu trọng số mô hình tốt nhất
best_acc = 0.0
model_save_path = 'best_model.pth'

# Callback: Early Stopping
patience = 5
early_stop_counter = 0



Number of classes: 10


In [15]:
# Huấn luyện mô hình
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        logits, _ = model(inputs)  # Chỉ lấy logits
        
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total

    # Đánh giá mô hình trên tập kiểm tra
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits, _ = model(inputs)  # Chỉ lấy logits
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        test_accuracy = 100. * correct / total
    
    # Điều chỉnh tốc độ học
    scheduler.step(train_loss)

    # In thông tin sau mỗi epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")


Shape before flattening: torch.Size([29, 512, 5, 5])
Shape after flattening: torch.Size([29, 12800])
Shape before flattening: torch.Size([8, 512, 5, 5])
Shape after flattening: torch.Size([8, 12800])
Epoch 1/50, Loss: 2.2615, Train Accuracy: 13.79%, Test Accuracy: 12.50%
Shape before flattening: torch.Size([29, 512, 5, 5])
Shape after flattening: torch.Size([29, 12800])
Shape before flattening: torch.Size([8, 512, 5, 5])
Shape after flattening: torch.Size([8, 12800])
Epoch 2/50, Loss: 2.1481, Train Accuracy: 13.79%, Test Accuracy: 12.50%
Shape before flattening: torch.Size([29, 512, 5, 5])
Shape after flattening: torch.Size([29, 12800])
Shape before flattening: torch.Size([8, 512, 5, 5])
Shape after flattening: torch.Size([8, 12800])
Epoch 3/50, Loss: 2.0164, Train Accuracy: 13.79%, Test Accuracy: 25.00%
Shape before flattening: torch.Size([29, 512, 5, 5])
Shape after flattening: torch.Size([29, 12800])
Shape before flattening: torch.Size([8, 512, 5, 5])
Shape after flattening: torch.S

In [16]:
import os
import torch

# Đặt đường dẫn tệp
path = 'resnet.pth'

# Tạo tệp rỗng nếu chưa tồn tại
if not os.path.exists(path):
    with open(path, 'w') as file:
        pass

# Lưu trọng số của mô hình vào tệp
torch.save(model.state_dict(), path)
print(f"Model weights saved to {path}")


Model weights saved to resnet.pth


In [24]:
# In ra các giá trị nhãn và số lượng lớp
print(f"Class labels: {set(classNames)}")
print(f"Number of classes: {num_classes}")

# Kiểm tra nhãn đã mã hóa
print(f"Max label in y_train_encoded: {max(y_train_encoded)}")
print(f"Max label in y_test_encoded: {max(y_test_encoded)}")


Class labels: {'Haerin', '2274601080019', 'Danielle', '2274601080005', 'Iroha', '2274601080013', '2274601080006', 'Minji', 'Hyein', 'Hanni'}
Number of classes: 10
Max label in y_train_encoded: 9
Max label in y_test_encoded: 9


In [23]:

# Khởi tạo mô hình với số lớp và độ xám
num_classes = len(data_loader.dataset.classes) # Thay đổi tùy theo dữ liệu của bạn
grayscale = False  # Thay đổi tùy theo dữ liệu của bạn
model = resnet34(num_classes, grayscale)

# Tải trọng số từ tệp
model_save_path = 'resnet.pth'
model.load_state_dict(torch.load(model_save_path))

# Đưa mô hình về chế độ đánh giá
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
# Định nghĩa hàm preprocess_image
def preprocess_image(image_path, image_size=(224, 224)):
    if not os.path.isfile(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Error reading image file: {image_path}")

    img_resized = cv2.resize(img, image_size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_tensor = torch.tensor(img_rgb, dtype=torch.float32).permute(2, 0, 1) / 255.0
    img_tensor = img_tensor.unsqueeze(0)
    
    return img_tensor

# Định nghĩa hàm predict_image
def predict_image(model, img_tensor):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        logits, _ = model(img_tensor)
        _, predicted_class = torch.max(logits, 1)
    
    return predicted_class.item()

# Định nghĩa hàm show_prediction
def show_prediction(image_path, predicted_class, base_dir):
    root = tk.Tk()
    root.title("Dự Đoán Hình Ảnh")
    root.geometry("800x600")
    
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    img_tk = ImageTk.PhotoImage(image=img_pil)
    
    lbl_img = tk.Label(root, image=img_tk)
    lbl_img.pack()
    
    txt = scrolledtext.ScrolledText(root, width=60, height=10, font=('Arial', 12))
    txt.pack()
    
    # Dự đoán lớp
    folder_name = str(predicted_class)
    if folder_name in classNames:
        index = classNames.index(folder_name)
        name = classNames1[index]
        info = classInfo.get(name, "Thông tin không có")
    else:
        name = "Không xác định"
        info = "Thông tin không có"
    
    txt.insert(tk.END, f"Dự đoán: Lớp {predicted_class}\nTên: {name}\nThông tin: {info}\n")
    
    lbl_img.image = img_tk

    root.mainloop()

# Ví dụ sử dụng
image_path = 'Hanni.jpg'
try:
    img_tensor = preprocess_image(image_path)
    predicted_class = predict_image(model, img_tensor)
    show_prediction(image_path, predicted_class, path)
except Exception as e:
    print(f"Error: {e}")

Shape before flattening: torch.Size([1, 512, 5, 5])
Shape after flattening: torch.Size([1, 12800])


In [16]:
import cv2
import numpy as np
import torch
import os

def preprocess_image(image_path, image_size=(224, 224)):
    if not os.path.isfile(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")

    # Đọc hình ảnh từ file
    img = cv2.imread(image_path)
    
    if img is None:
        raise ValueError(f"Error reading image file: {image_path}")
    
    print(f"Original image shape: {img.shape}")

    # Thay đổi kích thước hình ảnh
    img_resized = cv2.resize(img, image_size)
    
    # Chuyển đổi hình ảnh từ BGR (OpenCV) sang RGB
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    
    # Chuyển đổi hình ảnh thành tensor
    img_tensor = torch.tensor(img_rgb, dtype=torch.float32).permute(2, 0, 1) / 255.0
    img_tensor = img_tensor.unsqueeze(0)  # Thêm batch dimension
    
    return img_tensor


In [17]:
def predict_image(model, img_tensor):
    # Chuyển mô hình sang thiết bị (CPU/GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        logits, probas = model(img_tensor)
        _, predicted_class = torch.max(logits, 1)
    
    return predicted_class.item(), probas[0, predicted_class].item()


In [18]:
import tkinter as tk
from tkinter import scrolledtext
from PIL import Image, ImageTk

def show_prediction(image_path, predicted_class, base_dir):
    root = tk.Tk()
    root.title("Dự Đoán Hình Ảnh")
    root.geometry("800x600")
    
    # Đọc và hiển thị hình ảnh
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    img_tk = ImageTk.PhotoImage(image=img_pil)
    
    lbl_img = tk.Label(root, image=img_tk)
    lbl_img.pack()
    
    # Hiển thị kết quả dự đoán
    txt = scrolledtext.ScrolledText(root, width=60, height=10, font=('Arial', 12))
    txt.pack()
    
    # Tìm thư mục tương ứng với lớp dự đoán
    folder_path = get_folder_path_from_class_name(predicted_class, base_dir)
    if folder_path:
        class_info = load_class_info_from_folder(folder_path)
    else:
        class_info = "Thông tin không có"
    
    txt.insert(tk.END, f"Dự đoán: Lớp {predicted_class}\nThông tin: {class_info}\n")
    
    # Lưu tham chiếu đến đối tượng ImageTk
    lbl_img.image = img_tk

    root.mainloop()


In [19]:
def get_folder_path_from_class_name(class_index, base_dir):
    folder_name = str(class_index)
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):
        return folder_path
    return None

def load_class_info_from_folder(folder_path):
    info_file_path = os.path.join(folder_path, 'info.txt')
    if os.path.isfile(info_file_path):
        with open(info_file_path, 'r') as file:
            return file.read()
    return "Thông tin không có"


In [20]:
image_path = 'Hanni.jpg'
base_dir = 'student_images'

try:
    img_tensor = preprocess_image(image_path)
    predicted_class = predict_image(model, img_tensor)
    show_prediction(image_path, predicted_class, base_dir)
except Exception as e:
    print(f"Error: {e}")


Original image shape: (780, 624, 3)
Shape before flattening: torch.Size([1, 512, 5, 5])
Shape after flattening: torch.Size([1, 12800])


## Nhận dạng bằng ảnh

In [21]:
# Đọc hình ảnh từ file để nhận diện
test_image_path = 'Hannie.jpg'
img = cv2.imread(test_image_path)

# Thay đổi kích thước hình ảnh xuống còn 1/4 kích thước để xử lý nhanh hơn
imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)

# Chuyển đổi hình ảnh từ BGR (định dạng OpenCV) sang RGB (định dạng face_recognition)
imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

# Tìm tất cả các vị trí khuôn mặt trong hình ảnh
faces_in_frame = face_recognition.face_locations(imgS)

# Mã hóa các khuôn mặt được tìm thấy trong hình ảnh
encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)

# Tạo cửa sổ tkinter
root = tk.Tk()
root.title("Thông tin Sinh Viên")
root.geometry("1920x1080")

# Tạo vùng văn bản có thể cuộn
txt = scrolledtext.ScrolledText(root, width=40, height=20, font=('Arial', 12))  # Sử dụng font hỗ trợ Unicode
txt.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Tạo một nhãn để hiển thị hình ảnh
lbl = tk.Label(root)
lbl.pack(side=tk.LEFT)

# Biến lưu trữ số lượng khuôn mặt nhận diện
num_detected_faces = 0

# Lặp qua từng khuôn mặt được mã hóa và vị trí của nó
for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
    # So sánh khuôn mặt được mã hóa với các khuôn mặt đã biết
    matches = face_recognition.compare_faces(encoded_face_train, encode_face)
    
    # Tính khoảng cách giữa khuôn mặt được mã hóa và các khuôn mặt đã biết
    faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
    
    # Tìm chỉ số của khoảng cách nhỏ nhất (khuôn mặt giống nhất)
    matchIndex = np.argmin(faceDist)
    
    # Nếu tìm thấy sự phù hợp, tiến hành chú thích
    if matches[matchIndex]:
        name = classNames[matchIndex].upper()
        name1 = classNames1[matchIndex].upper()
        
        # Trích xuất vị trí khuôn mặt và mở rộng lại kích thước ban đầu
        y1, x2, y2, x1 = faceloc
        y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
        
        # Vẽ một hình chữ nhật xung quanh khuôn mặt
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Vẽ một hình chữ nhật đầy màu dưới khuôn mặt để hiển thị tên
        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        
        # Đặt tên của người đó dưới khuôn mặt
        cv2.putText(img, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
        
        # Lấy thời gian hiện tại
        current_time = datetime.now().strftime(f"- Ngày %d-%m-%Y \n - Giờ %H:%M:%S")
        # Hiển thị thông tin từ tệp .txt trong giao diện tkinter

        info = classInfo.get(classNames1[matchIndex])
        # Hiển thị thông tin từ tệp .txt và thời gian nhận diện trong giao diện tkinter
        txt.insert(tk.END, f" Tên: {name1} \n MSSV: {info} \n Nhận dạng lúc: \n {current_time}\n\n")
        
        # Tăng biến đếm số lượng khuôn mặt nhận diện
        num_detected_faces += 1

# Kiểm tra nếu không có khuôn mặt nào được nhận diện
if num_detected_faces == 0:
    txt.insert(tk.END, "Không nhận diện được khuôn mặt nào trong ảnh.\n")

# Chuyển đổi hình ảnh từ BGR sang RGB để sử dụng với PIL
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_pil = Image.fromarray(img)
img_tk = ImageTk.PhotoImage(image=img_pil)

# Đặt hình ảnh vào nhãn
lbl.config(image=img_tk)
lbl.image = img_tk

# Khởi động giao diện tkinter
root.mainloop()


: 

## Nhận dạng bằng camera

In [ ]:
# Tạo cửa sổ tkinter
root = tk.Tk()
root.title("Thông tin Sinh Viên")
root.geometry("1920x1080")

# Tạo vùng văn bản có thể cuộn
txt = scrolledtext.ScrolledText(root, width=40, height=20, font=('Arial', 12))  # Sử dụng font hỗ trợ Unicode
txt.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Tạo một nhãn để hiển thị hình ảnh
lbl = tk.Label(root)
lbl.pack(side=tk.LEFT)

# Biến điều khiển vòng lặp video
stop_video = threading.Event()

# Hàm xử lý từng khung hình từ camera
def process_frame(frame):
    # Thay đổi kích thước hình ảnh xuống còn 1/4 kích thước để xử lý nhanh hơn
    imgS = cv2.resize(frame, (0, 0), None, 0.25, 0.25)

    # Chuyển đổi hình ảnh từ BGR (định dạng OpenCV) sang RGB (định dạng face_recognition)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # Tìm tất cả các vị trí khuôn mặt trong hình ảnh
    faces_in_frame = face_recognition.face_locations(imgS)

    # Mã hóa các khuôn mặt được tìm thấy trong hình ảnh
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)

    # Biến lưu trữ số lượng khuôn mặt nhận diện
    num_detected_faces = 0

    # Lặp qua từng khuôn mặt được mã hóa và vị trí của nó
    for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
        # So sánh khuôn mặt được mã hóa với các khuôn mặt đã biết
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)

        # Tính khoảng cách giữa khuôn mặt được mã hóa và các khuôn mặt đã biết
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)

        # Tìm chỉ số của khoảng cách nhỏ nhất (khuôn mặt giống nhất)
        matchIndex = np.argmin(faceDist)

        # Nếu tìm thấy sự phù hợp, tiến hành chú thích
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            name1 = classNames1[matchIndex].upper()

            # Trích xuất vị trí khuôn mặt và mở rộng lại kích thước ban đầu
            y1, x2, y2, x1 = faceloc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

            # Vẽ một hình chữ nhật xung quanh khuôn mặt
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Vẽ một hình chữ nhật đầy màu dưới khuôn mặt để hiển thị tên
            cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)

            # Đặt tên của người đó dưới khuôn mặt
            cv2.putText(frame, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            # Lấy thời gian hiện tại
            current_time = datetime.now().strftime(f"- Ngày %d-%m-%Y \n - Giờ %H:%M:%S")

            # Hiển thị thông tin từ tệp .txt và thời gian nhận diện trong giao diện tkinter
            info = classInfo.get(classNames1[matchIndex])
            txt.insert(tk.END, f" Tên: {name1} \n MSSV: {info} \n Nhận dạng lúc: \n {current_time}\n\n")

    # Chuyển đổi hình ảnh từ BGR sang RGB để sử dụng với PIL
    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img_tk = ImageTk.PhotoImage(image=img_pil)

    # Đặt hình ảnh vào nhãn
    lbl.config(image=img_tk)
    lbl.image = img_tk

# Hàm lấy khung hình từ camera và gọi hàm xử lý khung hình
def video_loop():
    while not stop_video.is_set():
        ret, frame = cap.read()
        if ret:
            process_frame(frame)
        root.update_idletasks()
        root.update()

# Hàm lắng nghe phím nhấn 'q' để thoát
def listen_for_key():
    keyboard.add_hotkey('q', stop_video.set)

# Mở camera
cap = cv2.VideoCapture(0)

# Tạo và bắt đầu các luồng
video_thread = threading.Thread(target=video_loop)
key_listener_thread = threading.Thread(target=listen_for_key)

video_thread.start()
key_listener_thread.start()

# Khởi động giao diện tkinter
root.mainloop()

# Đợi cho các luồng kết thúc
video_thread.join()
key_listener_thread.join()

# Giải phóng camera khi kết thúc
cap.release()
cv2.destroyAllWindows()


# Camera cũ

In [ ]:
# Khởi tạo các biến
images = []
classNames = []
classInfo = {}  # Từ điển để lưu thông tin từ tệp .txt
classNames1 = []

# Đường dẫn đến thư mục chứa hình ảnh và tệp tin

# Duyệt qua các thư mục và tệp tin trong `path`
for root, dirs, files in os.walk(path):
    for cl in dirs:  # Duyệt qua các thư mục con
        class_path = os.path.join(root, cl)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            if img_name.endswith(('png', 'jpg', 'jpeg')):
                curImg = cv2.imread(img_path)
                if curImg is not None:
                    images.append(curImg)
                    txt_path = os.path.join(class_path, cl + '.txt')
                    if os.path.exists(txt_path):
                        with open(txt_path, 'r', encoding='utf-8') as file:
                            content = file.read().strip().splitlines()
                            name = content[0].strip()  # Lấy tên từ dòng đầu tiên
                            info = '\n'.join(content[1:]).strip()  # Lấy thông tin từ các dòng còn lại
                            classNames.append(cl)
                            classNames1.append(name)
                            classInfo[name] = info
                    else:
                        print(f"Error: Could not find the text file {txt_path}")
                else:
                    print(f"Error: Could not load image {img_path}")

# Hiển thị số lượng hình ảnh đã được nạp
print(f"Loaded {len(images)} images and corresponding names and info.")

# Mã hóa tất cả các khuôn mặt trong images và lưu vào danh sách
encoded_face_train = [face_recognition.face_encodings(img)[0] for img in images if len(face_recognition.face_encodings(img)) > 0]

# Kiểm tra nếu không có khuôn mặt nào được mã hóa
if not encoded_face_train:
    print("No faces were encoded. Exiting program.")
    exit()

# Tạo cửa sổ tkinter
root = tk.Tk()
root.title("Thông tin Sinh Viên")
root.geometry("1920x1080")

# Tạo vùng văn bản có thể cuộn
txt = scrolledtext.ScrolledText(root, width=40, height=20, font=('Arial', 12))  # Sử dụng font hỗ trợ Unicode
txt.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Tạo một nhãn để hiển thị hình ảnh
lbl = tk.Label(root)
lbl.pack(side=tk.LEFT)

# Biến điều khiển vòng lặp video
stop_video = threading.Event()

# Hàm xử lý từng khung hình từ camera
def process_frame(frame):
    # Thay đổi kích thước hình ảnh xuống còn 1/4 kích thước để xử lý nhanh hơn
    imgS = cv2.resize(frame, (0, 0), None, 0.25, 0.25)

    # Chuyển đổi hình ảnh từ BGR (định dạng OpenCV) sang RGB (định dạng face_recognition)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # Tìm tất cả các vị trí khuôn mặt trong hình ảnh
    faces_in_frame = face_recognition.face_locations(imgS)

    # Mã hóa các khuôn mặt được tìm thấy trong hình ảnh
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)

    # Biến lưu trữ số lượng khuôn mặt nhận diện
    num_detected_faces = 0

    # Lặp qua từng khuôn mặt được mã hóa và vị trí của nó
    for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
        # So sánh khuôn mặt được mã hóa với các khuôn mặt đã biết
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)

        # Tính khoảng cách giữa khuôn mặt được mã hóa và các khuôn mặt đã biết
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)

        # Tìm chỉ số của khoảng cách nhỏ nhất (khuôn mặt giống nhất)
        matchIndex = np.argmin(faceDist)

        # Nếu tìm thấy sự phù hợp, tiến hành chú thích
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            name1 = classNames1[matchIndex].upper()

            # Trích xuất vị trí khuôn mặt và mở rộng lại kích thước ban đầu
            y1, x2, y2, x1 = faceloc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

            # Vẽ một hình chữ nhật xung quanh khuôn mặt
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Vẽ một hình chữ nhật đầy màu dưới khuôn mặt để hiển thị tên
            cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)

            # Đặt tên của người đó dưới khuôn mặt
            cv2.putText(frame, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            # Lấy thời gian hiện tại
            current_time = datetime.now().strftime(f"- Ngày %d-%m-%Y \n - Giờ %H:%M:%S")
            
            # Hiển thị thông tin từ tệp .txt và thời gian nhận diện trong giao diện tkinter
            info = classInfo.get(classNames1[matchIndex])
            txt.insert(tk.END, f" Tên: {name1} \n MSSV: {info} \n Nhận dạng lúc: \n {current_time}\n\n")

    # Chuyển đổi hình ảnh từ BGR sang RGB để sử dụng với PIL
    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img_tk = ImageTk.PhotoImage(image=img_pil)

    # Đặt hình ảnh vào nhãn
    lbl.config(image=img_tk)
    lbl.image = img_tk

# Hàm lấy khung hình từ camera và gọi hàm xử lý khung hình
def video_loop():
    while not stop_video.is_set():
        ret, frame = cap.read()
        if ret:
            process_frame(frame)
        root.update_idletasks()
        root.update()

# Hàm lắng nghe phím nhấn 'q' để thoát
def listen_for_key():
    keyboard.add_hotkey('q', stop_video.set)

# Mở camera
cap = cv2.VideoCapture(0)

# Tạo và bắt đầu các luồng
video_thread = threading.Thread(target=video_loop)
key_listener_thread = threading.Thread(target=listen_for_key)

video_thread.start()
key_listener_thread.start()

# Khởi động giao diện tkinter
root.mainloop()

# Đợi cho các luồng kết thúc
video_thread.join()
key_listener_thread.join()

# Giải phóng camera khi kết thúc
cap.release()
cv2.destroyAllWindows()


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet34, ResNet34_Weights

# Khởi tạo mô hình ResNet-34 với weights
weights = ResNet34_Weights.DEFAULT
model = resnet34(weights=weights)

# Số lớp đầu ra (tùy thuộc vào dữ liệu của bạn)
num_classes = len(datasets.ImageFolder('student_images/train').classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Chuyển mô hình sang thiết bị (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Định nghĩa criterion và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Kỹ thuật giảm tốc độ học
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Callback: Lưu trọng số mô hình tốt nhất
best_acc = 0.0
model_save_path = 'best_model.pth'

# Callback: Early Stopping
patience = 5
early_stop_counter = 0

# Định nghĩa biến đổi dữ liệu
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Khởi tạo DataLoader
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('student_images/train', transform=transform),
    batch_size=32,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('student_images/test', transform=transform),
    batch_size=32,
    shuffle=False
)

# Huấn luyện mô hình
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total

    # Đánh giá mô hình trên tập kiểm tra
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        test_accuracy = 100. * correct / total
    
    # Điều chỉnh tốc độ học
    scheduler.step(train_loss)
    
    # Callback: Lưu trọng số mô hình tốt nhất
    if test_accuracy > best_acc:
        best_acc = test_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Model weights saved to {model_save_path}")
        early_stop_counter = 0  # Reset counter if model improves
    else:
        early_stop_counter += 1
    
    # Callback: Early Stopping
    if early_stop_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

    # In thông tin sau mỗi epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")
